In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import ecc
import helper
import script
import tx

### Exercise 1

#### 1.1. Make [this test](/edit/session5/tx.py) pass
```
tx.py:TxTest::test_verify_input1
```

In [2]:
# Exercise 1.1

reload(tx)
run_test(tx.TxTest('test_verify_input1'))

.
----------------------------------------------------------------------
Ran 1 test in 0.173s

OK


In [3]:
from binascii import hexlify, unhexlify
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

tx_ins = []
prev_tx = unhexlify('0025bc3c0fa8b7eb55b9437fdbd016870d18e0df0ace7bc9864efc38414147c8')
tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=0,
            script_sig=b'',
            sequence=0xffffffff,
        ))
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.99*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.1*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)
sighash = SIGHASH_ALL
z = tx_obj.hash_to_sign(0, sighash)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + bytes([sighash])
sec = pk.point.sec()
script_sig = bytes([len(sig)]) + sig + bytes([len(sec)]) + sec
script_sig = bytes([len(script_sig)]) + script_sig
tx_obj.tx_ins[0].script_sig = Script.parse(script_sig)
hexlify(tx_obj.serialize())

b'0100000001c847414138fc4e86c97bce0adfe0180d8716d0db7f43b955ebb7a80f3cbc2500000000006b6a47304402203b4c0d6481e7dd40be41aff752a1b8d4843cd86f299a9f5e4c772fedc22295920220704f8082f36a947986be83b8b7d9ed9a6996dd2a5e6944b1ad9ff864e9670874012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02c09ee605000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000'

### Exercise 2

#### 2.1. Make [this test](/edit/session5/tx.py) pass
```
tx.py:TxTest:test_sign_input
```

In [4]:
# Exercise 2.1

reload(tx)
run_test(tx.TxTest('test_sign_input'))

.
----------------------------------------------------------------------
Ran 1 test in 0.384s

OK


### Exercise 3

#### 3.1. Send 0.5 TBTC to this address `mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf`

#### Go here to send your transaction: https://live.blockcypher.com/btc-testnet/pushtx/

In [5]:
# Exercise 3.1

from binascii import hexlify, unhexlify
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script
from script import Script
from tx import TxIn, TxOut, Tx


# create tx ins using previous transaction that got sent to you
tx_ins = []
prev_tx = unhexlify('7eae0e316a456efa21491460d454bcebc51201e4e5aa144ffa30a4bb50142642')
tx_ins.append(TxIn(
    prev_tx=prev_tx,
    prev_index=0,
    script_sig=b'',
    sequence=0xffffffff,
))
# create tx outs one to 'mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf' for 0.5 TBTC
#  and the rest to yourself (remember to include a fee!)
tx_outs = []
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.05*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.049*100000000),
    script_pubkey=p2pkh_script(h160),
))
# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)
# initialize your private key
private_key = PrivateKey(8675309)
# sign the input
tx_obj.sign_input(0, private_key, SIGHASH_ALL)
print(hexlify(tx_obj.serialize()).decode('ascii'))

010000000142261450bba430fa4f14aae5e40112c5ebbc54d460144921fa6e456a310eae7e000000006b483045022100b35e2e9368ebcfdec774b25b93f7c995dec14027d15081b2eef2a84e18b2cb5602201a8e92d2681f9de3081f2303491fb84058bbf491ea6cf35799cf2950f742a7b2012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02404b4c00000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88aca0c44a00000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac00000000


### Exercise 4

#### 4.1. Find the hash160 of the RedeemScript
```
5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [6]:
# Exercise 4.1

from binascii import hexlify, unhexlify
from helper import hash160

hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'

# unhexlify script
redeem_script = unhexlify(hex_redeem_script)
# hash160 result
h160 = hash160(redeem_script)
# hexlify to display
print(hexlify(h160))

b'74d691da1574e6b3c192ecfb52cc8984ee7b6c56'


In [7]:
from binascii import unhexlify
from helper import encode_base58_checksum

print(encode_base58_checksum(b'\x05'+unhexlify('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

3CLoMMyuoDQTPRD3XYZtCvgvkadrAdvdXh


### Exercise 5

#### 5.1. Make [these tests](/edit/session5/helper.py) pass
```
helper.py:HelperTest:test_p2pkh_address
helper.py:HelperTest:test_p2sh_address
```

In [8]:
# Exercise 5.1

reload(helper)
run_test(helper.HelperTest('test_p2pkh_address'))
run_test(helper.HelperTest('test_p2sh_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.026s

OK


In [9]:
from binascii import unhexlify
from helper import double_sha256

sha = double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
                    
print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [10]:
from binascii import hexlify, unhexlify
from ecc import S256Point, Signature
from helper import double_sha256

sha = double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
point = S256Point.parse(unhexlify('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
r = 0x00dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e
s = 0x5a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937
sig = Signature(r,s)
print(point.verify(z, sig))

True


### Exercise 6

#### 6.1. Validate the second signature of the first input

```
0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000
```

The sec pubkey of the second signature is:
```
03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71
```

The der signature of the second signature is:
```
3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022
```

The redeemScript is:
```
475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

#### 6.2. Make [this test](/edit/session5/tx.py) pass
```
tx.py:TxTest:test_verify_input2
```

In [11]:
# Exercise 6.1

from binascii import hexlify, unhexlify
from io import BytesIO
from ecc import S256Point, Signature
from helper import double_sha256, int_to_little_endian
from script import Script
from tx import Tx, SIGHASH_ALL

hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022'
hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'

hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000'
stream = BytesIO(unhexlify(hex_tx))

# parse the S256Point and Signature
point = S256Point.parse(unhexlify(hex_sec))
sig = Signature.parse(unhexlify(hex_der))
# parse the Tx
t = Tx.parse(stream)
# change the first input's scriptSig to redeemScript
t.tx_ins[0].script_sig = Script.parse(unhexlify(hex_redeem_script))
# get the serialization
ser = t.serialize()
# add the sighash (4 bytes, little-endian of SIGHASH_ALL)
ser += int_to_little_endian(SIGHASH_ALL, 4)
# double_sha256 the result
to_sign = double_sha256(ser)
# this interpreted is a big-endian number is your z
z = int.from_bytes(to_sign, 'big')
point.verify(z, sig)

True

In [12]:
# Exercise 6.2

reload(tx)
run_test(tx.TxTest('test_verify_input2'))

.
----------------------------------------------------------------------
Ran 1 test in 0.356s

OK
